In [ ]:
# import matplotlib
%matplotlib inline
# matplotlib.use('macosx')

from gdsfactory.generic_tech import get_generic_pdk

genericPDK = get_generic_pdk()

LAYER = genericPDK.layers
layer_views = genericPDK.layer_views
layer_stack = genericPDK.layer_stack
cross_sections = genericPDK.cross_sections


import glob
import importlib
import sys

import gdsfactory as gf
import matplotlib.pyplot as plt
import numpy as np
import sax

sys.path.append("/Users/vahid/Downloads/PhotonicsAI_Project")
import time


def list_python_files(directory):
    """Lists all Python files in the given directory using glob."""
    # Use glob to list all .py files, exclude __init__.py
    return [
        f.split("/")[-1][:-3]
        for f in glob.glob(f"{directory}/*.py")
        if not f.endswith("__init__.py")
    ]


def import_modules(module_names):
    """Imports the specified function from each module in the given package and returns them in a dictionary."""
    functions = {}
    for module_name in module_names:
        full_module_name = f"PhotonicsAI.KnowledgeBase.DesignLibrary.{module_name}"
        module = importlib.import_module(full_module_name)
        func = getattr(module, module_name)
        globals()[module_name] = func
        functions[module_name] = func
    return functions


def import_models(module_names):
    """Imports the specified function from each module in the given package and returns them in a dictionary."""
    models_dict = {}
    for module_name in module_names:
        full_module_name = f"PhotonicsAI.KnowledgeBase.DesignLibrary.{module_name}"
        module = importlib.import_module(full_module_name)
        func = module.get_model
        models_dict.update(func())
        # globals()[module_name] = func()
    return models_dict


module_names = list_python_files("../../KnowledgeBase/DesignLibrary/")
cells = import_modules(module_names)
all_models = import_models(module_names)

DemoPDK = gf.Pdk(
    name="DemoPDK",
    layers=LAYER,
    cross_sections=cross_sections,
    cells=cells,
    layer_views=layer_views,
)
DemoPDK.activate()

DemoPDK = gf.Pdk(
    name="DemoPDK",
    layers=LAYER,
    cross_sections=cross_sections,
    cells=cells,
    layer_views=layer_views,
)
DemoPDK.activate()

In [ ]:
# import matplotlib
%matplotlib inline
# matplotlib.use('macosx')


def error_fn(wav, **kwargs):
    lambda_0 = kwargs.get("lambda_0")
    fsr = kwargs.get("FSR")
    arg = (2 * np.pi * lambda_0**2) / (wav * fsr)
    T = 0.5 * (1 + np.cos(arg))
    return T


netlist1 = """
instances:
  N1:
    component: mzi_1x2_pindiode_cband
    settings:
      delta_length: 60
      length: 320
ports:
  o1: N1,o1
  o2: N1,o2
  o3: N1,o3
"""


c = gf.read.from_yaml(netlist1)
c.plot()

gf_netlist_dict_recursive = c.get_netlist(recursive=True)
# recursive_netlist = sax.RecursiveNetlist.model_validate(gf_netlist_dict_recursive)
required_models = sax.get_required_circuit_models(gf_netlist_dict_recursive)
# print('Required Models ==>', required_models)

_mzi, info = sax.circuit(gf_netlist_dict_recursive, all_models)

start_time = time.time()
for _i in range(100):
    wl = np.linspace(1.5, 1.6, 200)
    result = _mzi(wl=wl)
end_time = time.time()
print(f"np: {(end_time - start_time)} seconds")
# jax running on CPU is not faster

wl = np.linspace(1.5, 1.6, 200)
result = _mzi(wl=wl)
plt.figure()
plt.plot(wl, np.abs(result[("o1", "o2")]))
# plt.plot( wl, np.abs(result[('o1', 'o3')]) )

plt.plot(wl, error_fn(wl, lambda_0=1.55, FSR=0.008), "--")

In [ ]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Integer

# Assuming 'terrain' is your generated and normalized terrain array


# Define the objective function
def terrain_height(params):
    x, y = params
    return terrain[y, x]  # Access the height; note the order of indices


# Define the search space
space = [Integer(0, width - 1, name="x"), Integer(0, height - 1, name="y")]

# Run the optimization
result = gp_minimize(
    terrain_height,
    space,
    n_calls=20,
    verbose=False,
    noise=0.1**2,  # the noise level (optional)
    x0=[int(width / 2), int(height / 2)],
)

optimal_x, optimal_y = result.x
optimal_height = result.fun

print(f"Optimal location (x, y): ({optimal_x}, {optimal_y})")
print(f"Minimum terrain height at this location: {optimal_height}")